# Onboarding a Data Owner

In this notebook you will control the OM Authority agent to issue a credential attesting to another entities ability to participate in a PPML flow as a Data Owner. 

This is very similar to the previous notebooks involving the data scientist, with just a few slight changes that highlight an important aspect of initiating connections that we will use later.


## 1. Initialise your Agent


In [1]:
%autoawait
import time
import asyncio
from termcolor import colored,cprint
# I think this is because jupyter notebook also runs an event loop
import nest_asyncio
nest_asyncio.apply()
from aries_basic_controller.aries_controller import AriesAgentController

ADMIN_URL = "http://om-authority-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL)

# The location the controller spins up a service and listens for webhooks from the agent
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""

agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE)

IPython autoawait is `on`, and set to use `asyncio`


## 2. Start Webhook Server

You always need to do this if you want to receive webhooks from your agent. 

In [2]:
loop = asyncio.get_event_loop()
loop.run_until_complete(agent_controller.listen_webhooks())

## 3. Load schema and definition identifiers

This is for the data owner this time as defined in the notebook 3.

In [3]:
%store -r data_owner_schema_id
%store -r data_owner_cred_def_id

if data_owner_schema_id and data_owner_cred_def_id:
    print("Successfully loaded identifiers required to issue Data Owner credentials")
    print("Schema :", data_owner_schema_id)
    print("Cred Def :", data_owner_cred_def_id)

Successfully loaded identifiers required to issue Data Owner credentials
Schema : Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1
Cred Def : S4yxhygNRJ78ZTS93fThL3:3:CL:188835:default


## 4. Populate Credential Attributes

Again you can put whatever you would like in these attributes.

When desigining an SSI Application you would likely want to create a way to enter new attribute values for each credential issued, maybe through some form? We will not be doing this in this notebook.

In [4]:

name=input("Please enter a name for the data owner: ")
domain=input("Please enter the domain of the data: ")
credential_attributes = [
    {"name": "name", "value": name},
    {"name": "domain", "value": domain},
]
print(credential_attributes)

Please enter a name for the data owner:  Will
Please enter the domain of the data:  Health


[{'name': 'name', 'value': 'Will'}, {'name': 'domain', 'value': 'Health'}]


## 5. Define A Entry Point for Data Scientists

**ALERT: I hope your paying attention. This is different from the previous notebooks**

A nice pattern we can use when managing connections within an SSI application is assigning them meaning based on how the connection was created. For example your application might have a button for normal users to register, but it also might have alternative pathways for administrative staff to connect with the agent.

Understanding this and using it to define custom entry points, for example connections established through this entrypoint could be challenged to meet a certain authentication policy before being trusted. We will see this later on.

For now just review the function defined below:

In [5]:
# The data store for connections that entered via this entrypoint
dataowner_connections = []
def create_dataowner_invite():
    # Create Invitation
    invite = asyncio.get_event_loop().run_until_complete(agent_controller.connections.create_invitation())
    connection_id = invite["connection_id"]
    
    # Represent connection in your application
    dataowner_conn = {
        "connection_id": connection_id,
        ## Note we could also have is_trusted or any other properties we like
        "is_active": asyncio.Future()
    }
    
    ## Add to list of dataowner connections
    dataowner_connections.append(dataowner_conn)
    
    print("Connection ID", connection_id)
    print("Invitation - Copy The Object Below \n")
    print(invite["invitation"])
    print("\n\n ----------------------------------------")
    
    # Asyncio futures allow you to wait until at some point in the future it is ready at another point in the application.
    asyncio.get_event_loop().run_until_complete(dataowner_conn["is_active"])
                                               
    # We can set information in a future when it is ready
    if dataowner_conn["is_active"].result() == True:
    
        ## issue a credential
        asyncio.get_event_loop().run_until_complete(agent_controller.issuer.send_credential(connection_id, data_owner_schema_id, data_owner_cred_def_id, credential_attributes, trace=False))

    
    

## 6. Register Listeners - Review the Code

Now we know the entry point for dataowners, and the datastore we use to record a representation of their connection in we can use them within our handler functions.

The connections_handler has been changed accordingly

In [6]:

def cred_handler(payload):
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange ID {exchange_id}")
    print("Agent Protocol Role", role)
    print("Protocol State ", state )
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

## THIS IS DIFFERENT
def connections_handler(payload):
    print("Handle Connection Webhook Payload")
    connection_id = payload["connection_id"]
    print("Connection ID", connection_id)

    state = payload['state']
    print("State", state)
    
    ## We can check if the connection webhook we are handling is of a specific type
    for dataowner_conn in dataowner_connections:
        if connection_id == dataowner_conn["connection_id"]:
            print("This is a Data Owner connection")
            if state == 'response':
                # seems to need to wait here for a bit
                time.sleep(1)
                # Must send trust ping to move connection to active
                asyncio.get_event_loop().run_until_complete(agent_controller.messaging.trust_ping(connection_id, "hey"))

            if state == 'active':
        #         print('Connection {0} changed state to active'.format(connection_id))
                print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))
                
                print("Setting Data Owner Connection is_active future result to True")
                dataowner_conn["is_active"].set_result(True)

connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)


Subscribing too: issue_credential
Subscribing too: connections


## 7. Establish A Connection with the Data Owner

We will use the entrypoint function we defined earlier. Again you will copy the printed invitation across to the Data Owner notebook, which should be at port 8889.

Notice that when you run the below cell it does not complete immediately (See the **In [*]:** to the left of the cell). This is because it is waiting for the connection to become active. You will be unable to run any other cells in this notebook until it does.

In [7]:
create_dataowner_invite()

wallet None
Handle Connection Webhook Payload
Connection ID e8ce45a0-c00f-4355-948e-67b6a92a9e68
State invitation
Connection ID e8ce45a0-c00f-4355-948e-67b6a92a9e68
Invitation - Copy The Object Below 

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'fd1bd5bc-5d27-422f-bdaa-ac44625c145a', 'serviceEndpoint': 'https://b1584135fc27.ngrok.io', 'recipientKeys': ['CY59xQ1ToVqPJX6hj1fqCJzR4EshrSyjpfPA2ysMap9r'], 'label': 'OM_AUTHORITY'}


 ----------------------------------------
wallet None
Handle Connection Webhook Payload
Connection ID e8ce45a0-c00f-4355-948e-67b6a92a9e68
State request
This is a Data Owner connection
wallet None
Handle Connection Webhook Payload
Connection ID e8ce45a0-c00f-4355-948e-67b6a92a9e68
State response
This is a Data Owner connection
wallet None
Handle Connection Webhook Payload
Connection ID e8ce45a0-c00f-4355-948e-67b6a92a9e68
State active
This is a Data Owner connection
Connection e8ce45a0-c00f-4355-948e-67b6a92a9e68 changed st

## Continue in the Data Owner Notebook

Be sure to come back here to review the webhook messages your receive after accepting the invitation. Pretty handy how your OM Authority handled issuing the Data Owner their credential.

## End of Tutorial

Great, we learnt a lot here. In this notebook you saw how to create specific entrypoints that can be used to define different actions for different connections. This is a pattern we will see more of.

In the Data Owner notebook you should have seen a detailed breakdown of the issue-credential protocol messages output by the credential handler. **IF YOU HAVE NOT SEEN THIS YOU SHOULD GO BACK AND CHECK THE ACCEPT CONNECTION OUTPUT CELL IN THAT NOTEBOOK**

Before moving on be sure to terminate the controller.

In [8]:
await agent_controller.terminate()

## Continue to 7.

Part 7 of this notebook series is not in the OM Authority application (notebooks). It takes place in the Data Owner and Data Scientist notebooks as they establish a connection and mutually request proof from the other.

We will start in the Data Owner notebook 7.